In [ ]:
import pandas as pd
import numpy as np
import json
import os
import glob
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json
import PIL
import skimage.io
import seaborn as sn
from collections import Counter
import tensorflow as tf
import collections
from keras import models
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Conv2D
import warnings
warnings.filterwarnings("ignore")

In [ ]:
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

In [ ]:
train_dir = "../input/herbarium-2022-fgvc9/train_images/"
test_dir = "../input/herbarium-2022-fgvc9/test_images/"

with open("../input/herbarium-2022-fgvc9/train_metadata.json") as json_file:
    train_meta = json.load(json_file)
with open("../input/herbarium-2022-fgvc9/test_metadata.json") as json_file:
    test_meta = json.load(json_file)

# Exploration

In [ ]:
train_meta.keys()

In [ ]:
print('the number of images in the training set : ',
      len(train_meta['images']))
print('the number of annotations in the training set :',
      len(train_meta['annotations']))

In [ ]:
print('The total number of classes : ',
      len(set([annotation['category_id'] for annotation in
               train_meta['annotations']])))

In [ ]:
train_df = pd.DataFrame(train_meta['annotations'])
display(train_df)

In [ ]:
train_cat = pd.DataFrame(train_meta['categories'])
display(train_cat)

In [ ]:
train_img = pd.DataFrame(train_meta['images'])
train_img.columns = ['image_id', 'file_name', 'license']
display(train_img)

In [ ]:
train_gen = pd.DataFrame(train_meta['genera'])
train_gen.columns = ['genus_id', 'genus']
display(train_gen)

In [ ]:
# merge important data

train_df = train_df.merge(train_cat, on='category_id', how='outer')
train_df = train_df.merge(train_img, on='image_id', how='outer')
train_df = train_df.merge(train_gen, on='genus_id', how='outer')

In [ ]:
train_df.columns

In [ ]:
print(train_df.info())
display(train_df)

In [ ]:
test_df = pd.DataFrame(test_meta)
test_df.columns = ['file_name', 'image_id', 'license']
print(test_df.info())
display(test_df)

In [ ]:
train_df.to_csv('train_data.csv', index=False)
test_df.to_csv('test_data.csv', index=False)

In [ ]:
print("Total Unique Values for each columns:")
print("{0:10s} \t {1:10d}".format('train_df', len(train_df)))
for col in train_df.columns:
    print("{0:10s} \t {1:10d}".format(col, len(train_df[col].unique())))

In [ ]:
meta = train_df[['file_name', 'family', 'genus_id', 'category_id']]
fam = meta.family.unique().tolist()
meta.family = meta.family.map(lambda x: fam.index(x))
gen = meta.genus_id.unique().tolist()
meta.genus_id = meta.genus_id.map(lambda x: gen.index(x))
display(meta)

In [ ]:
def visualize_train_batch(paths, categories, families, genus, species):
    plt.figure(figsize=(16, 16))

    for ind, info in enumerate(zip(paths, categories, families,
                                   genus, species)):
        path, category, family, genus, species = info

        plt.subplot(2, 3, ind + 1)

        image = cv2.imread(os.path.join(train_dir, path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        plt.imshow(image)

        plt.title(
            f"FAMILY: {family} GENUS: {genus}\n{species}",
            fontsize=10,
        )
        plt.axis("off")

    plt.show()

In [ ]:
def visualize_by_id(df, _id=None):
    tmp = df.sample(6)
    if _id is not None:
        tmp = df[df["category_id"] == _id].sample(6)

    visualize_train_batch(
        tmp["file_name"].tolist(),
        tmp["scientificName"].tolist(),
        tmp["family"].tolist(),
        tmp["genus_x"].tolist(),
        tmp["species"].tolist(),
    )

In [ ]:
for i in range(4):
    visualize_by_id(train_df, i)

In [ ]:
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

In [ ]:
family_cnt = np.round((meta['family']
                       .value_counts(normalize=True) * 100).head(10))
hv.Bars(family_cnt[::-1]).opts(
    title="North America Flora Families",
    color="purple", xlabel="Flora Families",
    ylabel="Percentage",
    xformatter='%d%%') \
    .opts(opts.Bars(
        width=600,
        height=600,
        tools=['hover'],
        show_grid=True,
        invert_axes=True))


In [ ]:
color = plt.cm.Greens(np.linspace(0, 1, 20))
train_df["scientificName"].value_counts(). \
    sort_values(ascending=False).head(20). \
    plot.pie(y="category_id", colors=color,
             autopct="%0.1f%%")
plt.title("North America Flora Scientific Names")
plt.axis("off")
plt.show()


In [ ]:
ax = train_df['scientificName'].value_counts()[:20]. \
    plot.barh(figsize=(16, 8), color='green')
ax.set_title('North America Flora Scientific Names', size=18, color='orange')
ax.set_ylabel('Scientific Names', size=10)
ax.set_xlabel('Count', size=10)

In [ ]:
color = plt.cm.viridis(np.linspace(0, 1, 20))
train_df["family"].value_counts().sort_values(ascending=False). \
    head(20).plot.pie(y="category_id", colors=color,
                      autopct="%0.1f%%")
plt.title("North America Flora Families")
plt.axis("off")
plt.show()


In [ ]:
genus_cnt = np.round((train_df['genus_x'].
                      value_counts(normalize=True) * 100).head(10))
hv.Bars(genus_cnt[::-1]). \
    opts(title="North America Flora Genus",
         color="#33B2FF", xlabel="Flora Genus",
         ylabel="Percentage", xformatter='%d%%') \
    .opts(opts.Bars(width=600, height=600,
                    tools=['hover'], show_grid=True, invert_axes=True))


In [ ]:
color = plt.cm.winter(np.linspace(0, 1, 20))
train_df["species"].value_counts().sort_values(ascending=False). \
    head(20).plot.pie(y="category_id", colors=color,
                      autopct="%0.1f%%")
plt.title("North America Flora Species")
plt.axis("off")



## Preprocessing

In [ ]:
img = os.path.join(train_dir, '000/00/00000__001.jpg')

In [ ]:
cv2.imread(img)

In [ ]:
cv2.imread(img).shape

In [ ]:
img = cv2.imread(img)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
w = 299
h = 299
res_img = cv2.resize(img, (w, h), interpolation=cv2.INTER_AREA)


In [ ]:
res_img.shape

In [ ]:
fig = plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
plt.imshow(img)
plt.title("original size : {}".format(img.shape))
plt.subplot(1, 2, 2)

plt.imshow(res_img)
plt.title("Resized shape : {}".format(res_img.shape))
plt.suptitle("Changing the image size", fontsize=20)
plt.show()


In [ ]:
img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)

plt.hist(img_yuv.flatten(), bins=range(256))
plt.title('Luminance distribution')
plt.subplot(1, 2, 2)
plt.imshow(img_yuv)

plt.show()

In [ ]:
# Equalization
img_yuv[:, :, 0] = cv2.equalizeHist(img_yuv[:, :, 0])
img_equ = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)

In [ ]:
fig = plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
plt.hist(img_equ.flatten(), bins=range(256))
plt.title('Luminance distribution after equalization')
plt.subplot(1, 2, 2)

plt.imshow(img_equ[:, :, ::-1])
plt.title('Image after equalization')
plt.show()

In [ ]:
datagen = ImageDataGenerator(featurewise_center=False,
                             featurewise_std_normalization=False,
                             rotation_range=180,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range=0.2)

x = img
x = x.reshape((1,) + x.shape)

aug_ = datagen.flow(x)

In [ ]:
aug_images = [next(aug_)[0].astype(np.uint8) for i in range(12)]

fig, axes = plt.subplots(3, 4, figsize=(20, 20))
axes = axes.flatten()
for ig, ax in zip(aug_images, axes):
    ax.imshow(ig)
    ax.axis('off')

plt.suptitle("Data augmentation", fontsize=24)
plt.show()

## Models

In [ ]:
from tensorflow import keras
from tensorflow.keras import backend as K

# calcul du macro f1 score


def f1_m(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true * y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2 * p * r / (p + r + K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)  

In [ ]:
meta = meta[meta['category_id'].isin(meta['category_id'].
                                     value_counts().head(20).index)]

In [ ]:
meta['category_id'] = meta['category_id'].apply(str)

In [ ]:
meta

In [ ]:
train, test = train_test_split(meta,
                               test_size=0.25,
                               stratify=meta['category_id'],
                               random_state=42)


In [ ]:
train['category_id'].unique().shape[0]

In [ ]:
test['category_id'].unique().shape[0]

In [ ]:
model = Sequential()

# 1 block
model.add(Conv2D(filters=16, kernel_size=(3, 3),
                 padding='same', use_bias=False,
                 input_shape=(299, 299, 3),
                 activation='relu'))
model.add(Conv2D(filters=16, kernel_size=(3, 3),
                 padding='same', use_bias=False,
                 input_shape=(299, 299, 3),
                 activation='relu'))
model.add(BatchNormalization(scale=False))
model.add(MaxPooling2D(pool_size=(4, 4),
                       strides=(4, 4),
                       padding='same'))
model.add(Dropout(0.25))

# 2end block
model.add(Conv2D(filters=32, kernel_size=(3, 3),
                 padding='same', use_bias=False,
                 activation='relu'))
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same',
                 use_bias=False,
                 activation='relu'))
model.add(BatchNormalization(scale=False))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4),
                       padding='same'))
model.add(Dropout(0.25))

# 3d block
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same',
                 use_bias=False,
                 activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same',
                 use_bias=False,
                 activation='relu'))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(train['category_id'].unique().shape[0],
                activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', f1_m])

In [ ]:
train_datagen = ImageDataGenerator(featurewise_center=False,
                                   featurewise_std_normalization=False,
                                   rotation_range=180,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.2)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
hist_cnn1 = model.fit_generator(
    train_datagen.flow_from_dataframe(dataframe=train,
                                      directory=train_dir,
                                      x_col="file_name",
                                      y_col="category_id",
                                      target_size=(299, 299),
                                      batch_size=32,
                                      class_mode="categorical"),
    validation_data=train_datagen.flow_from_dataframe(
        dataframe=test,
        directory=train_dir,
        x_col="file_name",
        y_col="category_id",
        target_size=(299, 299),
        batch_size=32,
        class_mode="categorical"),
    epochs=20,
    steps_per_epoch=len(train) // 32,
    validation_steps=len(test) // 32,
    verbose=2)


In [ ]:
def plot_score_accu(history):
    plt.figure(figsize=(13, 5))
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'], color="orange")
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [ ]:
def plot_score_f1(history):
    plt.figure(figsize=(13, 5))
    plt.plot(history.history['f1_m'])
    plt.plot(history.history['val_f1_m'], color="orange")
    plt.title('model f1-score')
    plt.ylabel('f1')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [ ]:
plot_score_accu(hist_cnn1)

In [ ]:
plot_score_f1(hist_cnn1)

On remarque que le modèle continue d'apprendre !

Par la suite, on va se limiter uniquement à 100 classes. les capacités de calcul étant limitées, nous ne pourrons pas traitrer toutes les classes.

In [ ]:
meta = train_df[['file_name', 'family', 'genus_id', 'category_id']]
fam = meta.family.unique().tolist()
meta.family = meta.family.map(lambda x: fam.index(x))
gen = meta.genus_id.unique().tolist()
meta.genus_id = meta.genus_id.map(lambda x: gen.index(x))
display(m)

In [ ]:
meta = meta[meta['category_id'].isin(meta['category_id']
                                     .value_counts().head(100).index)]
meta['category_id'] = meta['category_id'].apply(str)

In [ ]:
train, test = train_test_split(meta,
                               test_size=0.25,
                               stratify=m['category_id'],
                               random_state=42)


In [ ]:
train['category_id'].unique().shape[0], test['category_id'].unique().shape[0]

In [ ]:
model = Sequential()

# 1 block
model.add(Conv2D(filters=16, kernel_size=(3, 3),
                 padding='same',
                 use_bias=False,
                 input_shape=(299, 299, 3),
                 activation='relu'))
model.add(Conv2D(filters=16, kernel_size=(3, 3),
                 padding='same',
                 use_bias=False,
                 input_shape=(299, 299, 3),
                 activation='relu'))
model.add(BatchNormalization(scale=False))
model.add(MaxPooling2D(pool_size=(4, 4),
                       strides=(4, 4),
                       padding='same'))
model.add(Dropout(0.25))

# 2end block
model.add(Conv2D(filters=32, kernel_size=(3, 3),
                 padding='same',
                 use_bias=False,
                 activation='relu'))
model.add(Conv2D(filters=32, kernel_size=(3, 3),
                 padding='same',
                 use_bias=False,
                 activation='relu'))
model.add(BatchNormalization(scale=False))
model.add(MaxPooling2D(pool_size=(4, 4),
                       strides=(4, 4),
                       padding='same'))
model.add(Dropout(0.25))

# 3d block
model.add(Conv2D(filters=64, kernel_size=(3, 3),
                 padding='same',
                 use_bias=False,
                 activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3),
                 padding='same',
                 use_bias=False,
                 activation='relu'))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(train['category_id'].unique().shape[0], activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', f1_m])

hist_cnn1 = model.fit_generator(
    train_datagen.flow_from_dataframe(dataframe=train,
                                      directory=train_dir,
                                      x_col="file_name",
                                      y_col="category_id",
                                      target_size=(299, 299),
                                      batch_size=32,
                                      class_mode="categorical"),
    validation_data=train_datagen.flow_from_dataframe(
        dataframe=test,
        directory=train_dir,
        x_col="file_name",
        y_col="category_id",
        target_size=(299, 299),
        batch_size=32,
        class_mode="categorical"),
    epochs=10,
    steps_per_epoch=len(train) // 32,
    validation_steps=len(test) // 32,
    verbose=2)


In [ ]:
plot_score_accu(hist_cnn1)

On voit que les performances sont assez faibles pour 100 classes !

In [ ]:
# Chargement de VGG-16  sans les couches fully-connected
from keras.applications.vgg16 import VGG16  # VGG16

imnet = VGG16(weights=None,
              include_top=False,
              input_shape=(299, 299, 3))

model = models.Sequential()
model.add(imnet)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(train['category_id'].unique().shape[0],
                       activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', f1_m])

imnet_train_datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=180,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input)

In [ ]:
hist_imnet = model.fit_generator(
    imnet_train_datagen.flow_from_dataframe(dataframe=train,
                                            directory=train_dir,
                                            x_col="file_name",
                                            y_col="category_id",
                                            target_size=(299, 299),
                                            batch_size=32,
                                            class_mode='categorical'),
    validation_data=imnet_train_datagen.flow_from_dataframe(
        dataframe=test,
        directory=train_dir,
        x_col="file_name",
        y_col="category_id",
        target_size=(299, 299),
        batch_size=32,
        class_mode='categorical'),
    epochs=10,
    steps_per_epoch=len(train) // 32,
    validation_steps=len(test) // 32,
    verbose=2)


In [ ]:
plot_score_accu(hist_imnet)

In [ ]:
plot_score_f1(hist_imnet)

In [ ]:
from keras.applications.vgg16 import VGG16  # VGG16

imnet = VGG16(weights=None,
              include_top=True,
              input_shape=(299, 299, 3),
              classes=train['category_id'].unique().shape[0])


imnet.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', f1_m])

imnet_train_datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=180,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input)

hist_imnet_ = imnet.fit_generator(
    imnet_train_datagen.flow_from_dataframe(dataframe=train,
                                            directory=train_dir,
                                            x_col="file_name",
                                            y_col="category_id",
                                            target_size=(299, 299),
                                            batch_size=32,
                                            class_mode='categorical'),
    validation_data=imnet_train_datagen.flow_from_dataframe(
        dataframe=test,
        directory=train_dir,
        x_col="file_name",
        y_col="category_id",
        target_size=(299, 299),
        batch_size=32,
        class_mode='categorical'),
    epochs=10,
    steps_per_epoch=len(train) // 32,
    validation_steps=len(test) // 32,
    verbose=2)


In [ ]:
plot_score_accu(hist_imnet_)

In [ ]:
plot_score_f1(hist_imnet_)

On remarque que **VGG16** donne de mauvais résultats dans tous les cas : en important les couches fully connected ou en entrainement uniquement le classifier, les résultats sont très mauvais.

In [ ]:
from tensorflow.keras.applications import EfficientNetB0

efficientnet = EfficientNetB0(include_top=True,
                              weights=None,
                              input_shape=(299, 299, 3),
                              classes=train['category_id'].unique().shape[0])

efficientnet.compile(loss='categorical_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy', f1_m])

efficientnet_datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=180,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input)

hist_effnet = efficientnet.fit_generator(
    efficientnet_datagen.flow_from_dataframe(dataframe=train,
                                             directory=train_dir,
                                             x_col="file_name",
                                             y_col="category_id",
                                             target_size=(299, 299),
                                             batch_size=32,
                                             class_mode='categorical'),
    validation_data=efficientnet_datagen.flow_from_dataframe(
        dataframe=test,
        directory=train_dir,
        x_col="file_name",
        y_col="category_id",
        target_size=(299, 299),
        batch_size=32,
        class_mode='categorical'),
    epochs=10,
    steps_per_epoch=len(train) // 32,
    validation_steps=len(test) // 32,
    verbose=2)


In [ ]:
plot_score_accu(hist_effnet)

In [ ]:
plot_score_f1(hist_effnet)

On remarque que **le EfficientNetB0** donne de bien meilleurs résultats que les modèles précédents. De plus le modèle semble continuer d'apprendre, ce qui signie qu'en augmentant le nombre de "epoch", on peut atteindre encore de meilleurs résultats.

In [ ]:
efficient_net = EfficientNetB0(include_top=False,
                               weights=None,
                               pooling='max',
                               input_shape=(299, 299, 3))

for layer in efficient_net.layers:
    layer.trainable = False  # on freeze les couches 

x = efficient_net.output
x = Dense(128, activation='relu')(x)
x = Dropout(0.25)(x)

effnet_pred = Dense(100, activation='softmax')(x)

effnet = Model(inputs=efficient_net.input,
               outputs=effnet_pred)

effnet.compile(optimizer="adam",
               loss="categorical_crossentropy",
               metrics=["accuracy", f1_m])

hist_effnet = effnet.fit_generator(
    efficientnet_datagen.flow_from_dataframe(dataframe=train,
                                             directory=train_dir,
                                             x_col="file_name",
                                             y_col="category_id",
                                             target_size=(299, 299),
                                             batch_size=32,
                                             class_mode='categorical'),
    validation_data=efficientnet_datagen.flow_from_dataframe(
        dataframe=test,
        directory=train_dir,
        x_col="file_name",
        y_col="category_id",
        target_size=(299, 299),
        batch_size=32,
        class_mode='categorical'),
    epochs=10,
    steps_per_epoch=len(train) // 32,
    validation_steps=len(test) // 32,
    verbose=2)

In [ ]:
plot_score_accu(hist_effnet)

In [ ]:
plot_score_f1(hist_effnet)

On remarque que quand on n'importe pas les couches **fully connected** de **EfficientNetB0**, on obtient de très muavais résultats.

In [ ]:
def model_hyp(hp):
    
    efficientnet = EfficientNetB0(
        include_top=True,
        weights=None,
        input_shape=(299, 299, 3),
        classes=train['category_id'].unique().shape[0])

    hp_learning_rate = hp.Choice(
        # recherche du meilleur parametre pour learning_rate
        name='learning_rate',
        values=[1e-2, 1e-3, 1e-4])

    efficientnet.compile(optimizer=keras.optimizers.Adam(
        learning_rate=hp_learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy", f1_m])

    return efficientnet



In [ ]:
import kerastuner as kt

tn = kt.RandomSearch(
    model_hyp,
    objective='val_accuracy',
    max_trials=5)  # tuner of kerastuner

stop_early = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=5)  # arrêter l'entrainement qd val_accuracy ne s'améliore plus

tn.search(efficientnet_datagen.flow_from_dataframe(dataframe=train,
                                                   directory=train_dir,
                                                   x_col="file_name",
                                                   y_col="category_id",
                                                   target_size=(299, 299),
                                                   batch_size=32,
                                                   class_mode='categorical'),
          validation_data=efficientnet_datagen.flow_from_dataframe(
              dataframe=test,
              directory=train_dir,
              x_col="file_name",
              y_col="category_id",
              target_size=(299, 299),
              batch_size=32,
              class_mode='categorical'),
          epochs=10,
          callbacks=[stop_early])


In [ ]:
best_params = tn.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
print('Best learning rate :', best_params.get('learning_rate'))

In [ ]:
model_effnet = tn.hypermodel.build(best_params)

In [ ]:
hist_model_effnet = model_effnet.fit_generator(
    efficientnet_datagen.flow_from_dataframe(dataframe=train,
                                             directory=train_dir,
                                             x_col="file_name",
                                             y_col="category_id",
                                             target_size=(299, 299),
                                             batch_size=32,
                                             class_mode='categorical'),
    validation_data=efficientnet_datagen.flow_from_dataframe(
        dataframe=test,
        directory=train_dir,
        x_col="file_name",
        y_col="category_id",
        target_size=(299, 299),
        batch_size=32,
        class_mode='categorical'),
    epochs=10,
    steps_per_epoch=len(train) // 32,
    validation_steps=len(test) // 32,
    verbose=2)

In [ ]:
plot_score_accu(hist_model_effnet)

In [ ]:
plot_score_f1(hist_model_effnet)

On voit donc que les perfomances du modèle **EfficientNetB0** sont encore mieux qu'auparavant. Il faudra l'entraîner sur un nombre d'epoque plus grand que 10 pour avoir de meilleurs résultats.

In [ ]:
plt.figure(figsize=(13, 10))
plt.ylabel('Accuracy')
plt.xlabel('epoch')

plt.plot(
    hist_cnn1.history['accuracy'],
    label='CNN - Mean accuracy: {:.2f}'.
    format(np.mean(hist_cnn1.history['accuracy'])))

plt.plot(
    hist_imnet_.history['accuracy'],
    label='VGG16 - Mean accuracy: {:.2f}'.
    format(np.mean(hist_imnet_.history['accuracy'])))

plt.plot(
    hist_effnet.history['accuracy'],
    label='EfficientNetB0 without layers'
    'fully connected - Mean accuracy: {:.2f}'.
    format(np.mean(hist_effnet.history['accuracy'])))

plt.plot(
    hist_model_effnet.history['accuracy'],
    label='EfficientNetB0 tuned - Mean accuracy: {:.2f}'.
    format(np.mean(hist_model_effnet.history['accuracy'])))

plt.title('Accuracy of differents models tested', fontsize=14)

plt.legend(loc='best')
plt.grid(True)
plt.show()



le modèle final choisi sera donc **EfficientNetB0 tuned**

In [ ]:
resnet_model = tf.keras.applications.ResNet50(
    weights=None,
    include_top=True,
    pooling='max',
    input_shape=(299, 299, 3),
    classes=train['category_id'].unique().shape[0])

resnset_model.compile(loss='categorical_crossentropy', 
                      optimizer='adam',
                      metrics=['accuracy', f1_m])

resnet_train_datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=180,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    preprocessing_function=tf.keras.applications.resnet.preprocess_input)

hist_resnet = model.fit_generator(
    resnet_train_datagen.flow_from_dataframe(dataframe=train,
                                             directory=train_dir,
                                             x_col="file_name",
                                             y_col="category_id",
                                             target_size=(299, 299),
                                             batch_size=32,
                                             class_mode='categorical'),
    validation_data=resnet_train_datagen.flow_from_dataframe(
        dataframe=test,
        directory=train_dir,
        x_col="file_name",
        y_col="category_id",
        target_size=(299, 299),
        batch_size=32,
        class_mode='categorical'),
    epochs=10,
    steps_per_epoch=len(train) // 32,
    validation_steps=len(test) // 32,
    verbose=2)
